In [1]:
!pip install ftfy regex tqdm torch
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch-lightning

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fprcyok2
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-fprcyok2
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620


In [2]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss



In [3]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv")


In [4]:
clean_df.head()

,Unnamed: 0,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%
0,0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...,float holding luohu baoan arrangement shenzhen...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...
1,1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...,girls childrens students years children gift t...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...
2,2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...,hou men women sm alternative sex qing tail fox...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...
3,3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...,boss chinese burnin patron lucky paintings wal...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...
4,4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...,thick distressed straight lgnace slacksmusic t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...


In [5]:
col_to_test = [c for c in clean_df.columns.values if "caption" in c ]

In [6]:
col_to_test

['caption',
 'caption_crop_first_70%',
 'caption_crop_last_70%',
 'caption_crop_middle_70%']

In [7]:
model,preprocess = clip.load(CONFIG.model_name,CONFIG.device)

In [8]:
clip_model = ALIPRODUCT_CLIP(model)

In [9]:
def test_feature(caption_col):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,CONFIG.test_batch_size,preprocess,CONFIG.global_random_state,test=True)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    faiss_index = faiss.IndexFlatIP(512)
    faiss_index_gpu = faiss.index_cpu_to_all_gpus(faiss_index)
    faiss_index_gpu.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),10)

    # top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    # top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    # top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    # top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return {"top_5_embed":top5_k_e,"top_5_index":top5_k_y_pred,"top_10_embed":top10_k_e,"top_10_index":top10_k_y_pred}

In [10]:
# def topk_accuracy(y_true, pred, label_map, k,threshold):
#     pred[pred < threshold] = 0
#     pred_count = []
#     remap = np.vectorize(lambda x: label_map[x]["label"])
#     #get top k values
#     top_k_pred = remap(pred.argsort(axis=1)[:, -k:][:, ::-1])
#     # append 1 if true label is in top k values else append 0
#     for i in range(len(y_true)):
#         if y_true[i] in top_k_pred[i]:
#             pred_count.append(1)
#         else:
#             pred_count.append(0)
#     return sum(pred_count)/len(pred_count)

In [11]:

# for col in col_to_test:
#     top5_acc,top10_acc = test_feature(col)
#     col_name.append(col)
#     top5_acc_preds.append(top5_acc)
#     top10_acc_preds.append(top10_acc)
# pred_df = {"caption_clean_method":col_name,"top_5":top5_acc_preds,"top_10":top10_acc_preds}

top_acc_preds = {}
for col in col_to_test:
    top_acc = test_feature(col)
    top_acc_preds[col] =top_acc 

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:   0%|          | 0/50 [00:12<?, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile


...

Predicting DataLoader 0:  46%|████▌     | 23/50 [01:15<01:28,  3.29s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:  50%|█████     | 25/50 [01:21<01:21,  3.26s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|██████████| 50/50 [02:36<00:00,  3.12s/it]


In [12]:
clip_model = ALIPRODUCT_CLIP(model)
checkpoint = torch.load("../aliproduct_run_1_ViT-B/32_val_clip_loss=0.85487.ckpt")
clip_model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [13]:
top_acc_preds_trained = {}
for col in col_to_test:
    top_acc_trained = test_feature(col)
    top_acc_preds_trained[col] = top_acc_trained


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:   0%|          | 0/50 [00:10<?, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Predicting DataLoader 0:  46%|████▌     | 23/50 [01:16<01:29,  3.32s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:  50%|█████     | 25/50 [01:22<01:22,  3.29s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|██████████| 50/50 [02:36<00:00,  3.14s/it]


In [14]:
def combine_results(all_model_preds):
    top_5_combine_results = []
    top_10_combine_results = []
    for feature in all_model_preds.keys():
        top_5_sim = all_model_preds[feature]["top_5_embed"]
        top_5_index = all_model_preds[feature]["top_5_index"]
        top_10_sim= all_model_preds[feature]["top_5_embed"]
        top_10_index = all_model_preds[feature]["top_5_index"]
        top_5 = np.array([dict(zip(l,p)) for l,p in zip(top_5_index,top_5_sim)])
        top_5_combine_results.append(top_5)
        top_10 = np.array([dict(zip(l,p)) for l,p in zip(top_10_index,top_10_sim)])
        top_10_combine_results.append(top_10)
    top_5_combine_results = np.stack(top_5_combine_results,axis=1)
    top_10_combine_results = np.stack(top_10_combine_results,axis=1)
    return top_5_combine_results,top_10_combine_results

           
top_5_combine_results,top_10_combine_results= combine_results(top_acc_preds)

top_5_combine_results_trained,top_10_combine_results_trained = combine_results(top_acc_preds_trained)





In [15]:
def get_blend_top_k(all_combine_pred,k):
    blend_result = []
    for row in tqdm(all_combine_pred):
        row_dic = {}
        for dic in row:
            for i in dic:
                if i in row_dic and dic[i] > row_dic[i] and len(row_dic)!=0:
                    row_dic[i] = dic[i]
                else:
                    row_dic[i] = dic[i]
        blend_result.append(sorted(row_dic,key=row_dic.get,reverse=True)[:k])
    return blend_result


In [16]:
top_5_blend_result = get_blend_top_k(top_5_combine_results,5)
top_10_blend_result = get_blend_top_k(top_10_combine_results,10)
top_5_blend_result_trained = get_blend_top_k(top_5_combine_results_trained,5)
top_10_blend_result_trained = get_blend_top_k(top_10_combine_results_trained,10)

100%|██████████| 50000/50000 [00:00<00:00, 69233.59it/s] 


In [17]:
top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(clean_df.index.values,top_5_blend_result)])
top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(clean_df.index.values,top_10_blend_result)])
top5_preds_trained = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(clean_df.index.values,top_5_blend_result_trained)])
top10_preds_trained = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(clean_df.index.values,top_10_blend_result_trained)])

In [18]:
top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
top5_acc_trained = top5_preds_trained[top5_preds_trained ==1].shape[0] / top5_preds_trained.shape[0]
top10_acc_trained = top10_preds_trained[top10_preds_trained ==1].shape[0] / top10_preds_trained.shape[0]

In [19]:
print("base accuracy")
print(top5_acc*100,"%")
print(top10_acc*100,"%")

base accuracy
4.664 %
7.542 %


In [20]:
print("trained accuracy")
print(top5_acc_trained*100,"%")
print(top10_acc_trained*100,"%")

trained accuracy
12.818 %
19.114 %
